In [ ]:
# !pip install cohere
# !pip install langchain openai
# !pip install unstructured chromadb
# !pip install tiktoken
# !pip install duckdb
# !pip install -U pandas

In [ ]:
import pandas as pd
import duckdb

import pandas as pd
pd.set_option('display.max_colwidth', 180) 
path = '/workspaces/codesandbox-template-blank/Code_for_Embeddings'

account_vulnerabilities_df=pd.read_csv(f'{path}/data/account_vulnerabilities.csv')
public_tools_df=pd.read_csv(f'{path}/data/public_tools.csv')
versions_data_df=pd.read_csv(f'{path}/data/versions_data.csv')
vulnerabilities_df=pd.read_csv(f'{path}/data/vulnerabilities.csv')
decisions_data_df=pd.read_csv(f'{path}/data/decisions_data.csv')
parameters_data_df=pd.read_excel(f'{path}/data/parameters.xlsx')

import duckdb
duckdb.default_connection.execute("SET GLOBAL pandas_analyze_sample=100000")
def sql_helper(sql):
    return duckdb.query(sql).to_df()

def sql_helper_one(sql):
    return duckdb.query(sql).fetchone()

#### Extras

In [ ]:
import pandas as pd
import duckdb
sql=f"""
    select *
    from public_tools_df
    limit 5
"""
sql_helper(sql)

In [ ]:
sql=f"""
    select * 
    from parameters_data_df
"""
df=sql_helper(sql)

def proc(x):
    for inx,item in x.Parameters.items():
        x.Parameters.iloc[inx]='_'.join(item.replace('-',' ').split(' '))
    return x.Parameters

df = df.assign(Formatted_Parameters=proc)

sql=f"""
    select Formatted_Parameters as Parameters,"Weightage " as Weightage
    from df
"""
sql_helper(sql).to_csv('parameters.csv',index=False)

#### Work

In [ ]:
sql=f"""
    select number,tool_id,tool_name,major_version_number,'' as max_version_number,
    0 as Freshness_Distance_from_latest_version,
    0 as tool_version_External_Factors_Freshness_EOL_Support,
    0 as tool_version_External_Factors_Last_commit_date_of_the_package,
    0 as tool_version_External_Factors_Security_Vulnerabilities,
    0 as tool_version_Internal_Factors_Adoption_Adoption_Stages,
    0 as tool_version_Internal_Factors_Adoption_Internal_Adoption_Skill_set_Availability,
    0 as tool_version_Internal_Factors_cost
    from versions_data_df
    where tool_id=2739
    order by number
"""
df_Freshness_Distance_from_latest_version=sql_helper(sql)

In [ ]:
df_Freshness_Distance_from_latest_version

#### Security Vulnerabilities

In [ ]:
import pandas as pd
import duckdb
sql=f"""
    select 
        * 
    from 
        vulnerabilities_df
    limit 5
"""
# df= sql_helper(sql)

sql_helper(sql)

#### Distance

In [ ]:
sql=f"""
    select vdd.number,vd.severity,vdd.tool_id
    from versions_data_df as vdd
    inner join vulnerabilities_df as vd
    on vdd.tool_id = vd.tool_id
    where vdd.tool_id=2739   
    order by vdd.number
"""
# sql=f"""
#     select max(vdd.tool_id)
#     from vulnerabilities_df as vdd
# """
sql_helper(sql)

In [ ]:
babel_version

In [ ]:
babel_version.to_csv('gd.csv')

In [ ]:
for inx,item in babel_version.iterrows():
    print(item.number)

In [ ]:
import os
from langchain.llms import AzureOpenAI,Cohere,OpenAI,GooglePalm

# llm = AzureOpenAI(
#     deployment_name="StackGPT35Turbo",
#     model_name="gpt-35-turbo",
#     temperature=0,
#     max_tokens=10 
# )
# llm = Cohere(model="command-xlarge-beta",temperature=0,max_tokens=10,k=0)
llm=GooglePalm(temperature=0,max_output_tokens=10,top_p=0.8)

In [ ]:
from langchain.prompts import PromptTemplate

def prompt_generation(current_tool_version,latest_tool_version):
    print(f'current_tool_version --> {current_tool_version}')
    template = f"""You are excellent AI assistant.You known to identify and quantize distance between two nodes.
    If we lable distance between in range of 1 - 5.
    that is if it's very near than label it as 1 and if it is too far
    it should label as 5. 
    For example

    if node1 is 1.0 and node2 is 2.0 than distance rank is 1.
    if node1 is 1.0 and node2 is 3.0 than distance rank is 2.
    if node1 is 1.0 and node2 is 4.0 than distance rank is 3.
    if node1 is 1.0 and node2 is 5.0 than distance rank is 4.
    if node1 is 5.0.0 and node2 is 5.2.0 than distance rank is 0.
    if node1 is 3.3.0 and node2 is 3.2.0 than distance rank is 0.
    if node1 is 4.4.0 and node2 is 5.2.0 than distance rank is 1.
    if node1 is 4.4.0-beta.8 and node2 is 5.5.0 than distance rank is 1.
    
    if node1 is 6.13.1 and node2 is 7.9.6 than distance rank is 1.
    if node1 is 6.14.0 and node2 is 7.9.6 than distance rank is 1.
    if node1 is 6.15.2 and node2 is 7.9.6 than distance rank is 1.
    if node1 is 6.21.0 and node2 is 7.9.6 than distance rank is 1.
    if node1 is 6.23.1 and node2 is 7.9.6 than distance rank is 1.
    if node1 is 6.3.1 and node2 is 7.9.6 than distance rank is 1.
    
    if node1 is 7.0.0-beta.54 and node2 is 7.9.6 than distance rank is 1.


        there is node1 is {current_tool_version} and node2 is {latest_tool_version}.

    now can you respond me what is the difference between node1 and node2 in terms of 
    rank between 1 to 5.
    Kindly give me response in one word only no need to verbose.
    Response:
    """
    return template

def convert_tool_version_External_Factors_Freshness_Distance_from_latest_version(response):
    print(f'Response --> {response}')
    internal_weitage=0
    rank=int(response)
    if rank >= 3 and rank <= 5:
        internal_weitage = 50
    elif rank >= 1 and rank <= 2: 
        internal_weitage = 90
    else:
        internal_weitage = 100

    return((internal_weitage/100)*5) 

In [ ]:
template = prompt_generation("6.18.2","7.9.6")
response=llm(template)
convert_response_to_score=convert_tool_version_External_Factors_Freshness_Distance_from_latest_version(response)
convert_response_to_score

In [ ]:
df_Freshness_Distance_from_latest_version.columns

In [ ]:

import time
for i,row in df_Freshness_Distance_from_latest_version.iterrows():
    df_Freshness_Distance_from_latest_version.at[i,'max_version_number'] = "7.9.6"
    # print(f'data --> {row.number}')    
    # template = prompt_generation(row.number,"7.9.6")
    # response=llm(template)
    # df_Freshness_Distance_from_latest_version.at[i,'Freshness_Distance_from_latest_version']=convert_tool_version_External_Factors_Freshness_Distance_from_latest_version(response)
    time.sleep(3)

df_Freshness_Distance_from_latest_version

In [ ]:
df_Freshness_Distance_from_latest_version.to_excel("result.xlsx")